# $SPHINCS^+$ Python : Example #

### Importing $SPHINCS^+$ Class ###

In [1]:
from package.sphincs import Sphincs
from tqdm import tqdm

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

By default, $SPHINCS^+$ parameters are set like:
   - Security Parameter: $n=16$
   - Winternitz Parameter: $w=16$ (Should be set as 4, 16 or 256)
   - Hypertree Height: $h=64$
   - Hypertree Layers: $d=8$ ($d|h$)
   - $FORS$ Trees Number: $k=10$
   - $FORS$ Trees Height: $a=15$

Changes must be done before keys generations, signing and verification

In [2]:
sphincs = Sphincs()

sphincs.set_winternitz(4)
# Or
sphincs.set_w(4)

sphincs.set_hypertree_height(32)

### Generating a Key Pair ### 

In [3]:
sk, pk = sphincs.generate_key_pair()
print("Secret Key: ", sk)
print()
print("Public Key: ", pk)

Secret Key:  b'\x8d\xec(5\xd9yE\x9cA]\x9f{5\xc1(\xdc\xa6a\x9d\x8fe!\x14He\xbe+g`\xf7\xde\xad\x7f,\xbag\x0bj\xd7\x01\xfaz\x00\x89\xab\xa7u\xbaq)\xec\xc6\xe9bK\xab\x03\xc9\xc5%\x18\xa8\x02\x04\xe3\x98lU\xa3h\xf6W&\\i\x00i\rx\x96\xcb\xb2\xf3W\x18\rD4\xc2M\xc6\xf4\xd0\x91=\x86\x95\xc81\xba\xa4u\x1b\x12\xd2\xca\xbc\xfbV#^\xb6\x8f\x80Q$\x0c\x83\xc4\xdb\x87)\xbc<\xa5\x84\xa7\x01'

Public Key:  b'\xe3\x98lU\xa3h\xf6W&\\i\x00i\rx\x96\xcb\xb2\xf3W\x18\rD4\xc2M\xc6\xf4\xd0\x91=\x86\x95\xc81\xba\xa4u\x1b\x12\xd2\xca\xbc\xfbV#^\xb6\x8f\x80Q$\x0c\x83\xc4\xdb\x87)\xbc<\xa5\x84\xa7\x01'


### Signing a Message ###

The message must be in a $bytes()$ format

In [4]:
# m = b"No one knows the reason for all this, but it is probably quantum. - Pyramids, Terry Pratchett (1989)"
m = b'Ripples of paradox spread out across the sea of causality.'
print("Message to be signed: ", m)

Message to be signed:  b'Ripples of paradox spread out across the sea of causality.'


In [5]:
signature = sphincs.sign(m, sk)

In [6]:
signature

b'|\xc9\\i+\x85\xaf\xa7\xbb=\xd8g\x8fE.,\xb3p\xfc0\x07\xfa\x0b3\x8f\x113M\xa9\xff\'S\xe0V\x91w\xb5\x84\xe7\xbf\x8e\x82X\x0bh\xcc\xf5\xdd\xcf\xfa\x17/\x0c0\x0b\xc5\xed\xc8\xc9\x04\xc0\xb0D\xa7\xfet\xf9\x974\xcbg\x8bOB\x96\x01/\xf1C\xfaK\x8ey\x08\xd4U\xf4\xd7\x12S\'\x0b\xcb\x8a@l\x1c\xdc\xdd\xce\xbf\x9enT\xc4\x8fayC\xce\x19\x9c4\x9b\x90\xe9\x97\x91\xb3\x9d\x89\x9d\xe9\x94\x12\xef\xb7\xe7\xd7\xee$\xd4\xedC\xa9<\xcal\xe4\xaf\xa2\xae\xa4_u\xb1\xe7*z\xe9DM\x15\x85t\xaa\x1c\xde\xd19\x8b\xb5\xde\x94\x98\x9d\xfb.\xa1M\xe2ZUql\x89\x88d\xf8\xbc:\x91B\xc0\xe7{\xbb\xfc\x939I3\xdfZ;,$\xa1\xbb\xd5\x92\x9e\xbd}\x00GAs\xfcV\x9a\xa0t\xf6p\xd9NH\x9b\xe7\x97\xcb\xf9\x81\x19F\xd2\x9e\xfe\x1f\x0f\x14\x03\x86\xb6\xb2\xbb*\xa1\x02f\xba\x1bP\xe9\x16\x0f&\x98hy\x80\x81\xb0\xca\xf4P\xb8\xa5\x965\x98\x81\xc5\xba\x1ag\xce\xcc\xa8\xfb\xad\xdb\xda\xef\n\'yB\xec\x84\xcel,\xd6\x8a\xec\xbc\xa3\x04\xee\xcfUK\x86@\x92\x8b\'\xb5\xbd\x85\xd6\x07v\xa6\xd3l\xc7\xa2i;\x1c\xc4h\xdeB`~%m\xfb\x0fP\x9bC\xbf\xc7{\xaeI\x9cn\xb5-\x1

In [7]:
print("Signature bytes-size: ", len(signature))

Signature bytes-size:  7684


### Verifying a Signature associated with a Message ###

In [8]:
print("Is signature correct ? ", sphincs.verify(m, signature, pk))

Is signature correct ?  True


### Trying to find secret key with a Brute Force Attack on Secret Key

In [11]:
sk_crack = bytes()

for i in range(0, 2 ** (sphincs._n * 8)):
    print(f"Value of i is {i}", end="\r")
    sk_crack = i.to_bytes(sphincs._n, 'big')  # Secret Key

    # Random Secret PRF, important to prevent forged messages from actual messages
    sk_crack += bytes(sphincs._n)
    # But Because we are brute forcing Secret Key, messages are forged before
    # We don't need to create a really random one (0 is fine)
    sk_crack += pk  # Public Key

    sig_crack = sphincs.sign(m, sk_crack)  # Creating a signature

    # Check if signature could be trust with the Public Key
    if sphincs.verify(sig_crack, m, pk):
        print("Secret Key Found: ", sk_crack,
              "\nWith main Private Seed: ", sk_crack[:sphincs._n])
        print("Cycles: ", i)
        break

print("\nDid we found the actual Private Seed? ",
      sk[:sphincs._n] == sk_crack[:sphincs._n])

if sk[:sphincs._n] != sk_crack[:sphincs._n]:
    print("We found a collision with the main seed!")


### Trying to forge message with found Key ###

In [ ]:
m2 = b'The pen is mightier than the sword ... if the sword is very short, and the pen is very sharp.'

signature2 = sphincs.sign(m2, sk_crack)

print(len(signature2))


In [ ]:
print("Is signature Correct ? ", sphincs.verify(signature2, m, pk))


Our signature is wrong because we tried finding the secret key using only $m$ signature, with $m_2$, this kind of collision doesn't work !
We need to find the real secret key in order to forge our own message, and so test every possibilities.

### Graft Fault Attack on $SPHINCS^+$ Framework

We will try to attack $SPHINCS^+$ framework by grafting a fake tree on the main hypertree by exploiting an error generated by the signing alogirthm as it can be explained in this document : https://eprint.iacr.org/2018/102.pdf

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

We will be using :
   - Security Parameter: $n=2$
   - Winternitz Parameter: $w=16$
   - Hypertree Height: $h=4$
   - Hypertree Layers: $d=2$
   - $FORS$ Trees Number: $k=4$
   - $FORS$ Trees Height: $a=2$

In [ ]:
sphincs = Sphincs()

# sphincs.set_n(2)
# sphincs.set_h(4)
# sphincs.set_d(2)
# sphincs.set_k(4)
# sphincs.set_a(2)

We will suppose that a message has been signed twice while a flaw in the algorithm made the second signature with a faulty authentification path with private key _b'mo\x9c\x1bS\xd4\x05\x16'_

In [ ]:
# Public key
pk = b'S\xd4\x05\x16'


In order to simplify this attack, me used the same $R$ variable so we could be sure our two signatures will use the same path of authentification

In [ ]:
m = b'Not our fault if it breaks...'
sig1 = b'\x00\x00QUS\xff\x90\x94uvD\xa1\xd8\xce\xa0\xa8\xd2\x8b`9\x84\x10\x07-i\x94\xfe3\xdb\x97\x00\xc8E\xd3\xdd\x14\x15\x81n\x13\x1f\x8d\xc3?\xee\xfe\xb0\xa0puV\xb6OEH;\xfe\xe1'
sig2 = b'\x00\x00QUS\xff\x90\x94uvD\xa1\xd8\xce\xa0\xa8\xd2\x8b`9\x84\x10\x07-i\x94\xfe3\xdb\x97\x00\xc8E\xd3\xdd\x14\x15\x81n\x13\x00\x00r\x05\xca\xb3P\xfd\x11\xb1\xbbN\r\xd8H;\xfe\xe1'


In [ ]:
# Splitting SIGs in order to find wots message and signature
h_prime = sphincs._h_prime
len_0 = sphincs._len_0
len_1 = sphincs._len_1
n = sphincs._n
d = sphincs._d
a = sphincs._a
k = sphincs._k
h = sphincs._h


def sigs_from_main_sig(sig):
    sig_tab = []

    sig_tab += [sig[:n]]  # R

    sig_tab += [[]]  # SIG FORS
    for i in range(n,
                   n + k * (a + 1) * n,
                   n):
        sig_tab[1].append(sig[i:(i + n)])

    sig_tab += [[]]  # SIG Hypertree
    for i in range(n + k * (a + 1) * n,
                   n + k * (a + 1) * n + (h + d * len_1) * n,
                   n):
        sig_tab[2].append(sig[i:(i + n)])
    return sig_tab


def sig_wots_from_sig_xmss(sig):
    return sig[0:len_1]


def auth_from_sig_xmss(sig):
    return sig[len_1:]


def sigs_xmss_from_sig_ht(sig):
    sigs = []
    for i in range(0, d):
        sigs.append(sig[i * (h_prime + len_1):(i + 1) * (h_prime + len_1)])

    return sigs


splitted_sig1 = sigs_from_main_sig(sig1)
splitted_sig2 = sigs_from_main_sig(sig2)
xmss_sig1 = sigs_xmss_from_sig_ht(splitted_sig1[2])
xmss_sig2 = sigs_xmss_from_sig_ht(splitted_sig2[2])

# WOTS signature from the XMSS tree d-1 is signing the previous auth from tree d-2
print("Auth XMSS d-2 sig1: ", auth_from_sig_xmss(xmss_sig1[0]))
print("WOTS SIG d-1  sig1: ", sig_wots_from_sig_xmss(xmss_sig1[1]))
print("Auth XMSS d-2 sig2: ", auth_from_sig_xmss(xmss_sig2[0]))
print("WOTS SIG d-1  sig2: ", sig_wots_from_sig_xmss(xmss_sig2[1]))
print()
print("Auth XMSS d-1 sig1: ", auth_from_sig_xmss(xmss_sig1[1]))
print("Auth XMSS d-1 sig2: ", auth_from_sig_xmss(xmss_sig2[1]))


As we can see, our second signature has a faulted authentification path, different from the first signature. Because of this, we obtain an other signature from the same $WOTS^+$ leaf. But even if the signature is different, because it has been successfully signed, it is counted as a true signature:

In [ ]:
print("Is sig1 correct? ", sphincs.verify(sig1, m, pk))
print("Is sig2 correct? ", sphincs.verify(sig2, m, pk))


We need to find a random _seed_ for which our forged message go through the same $WOTS^+$ leaf. To do this, we need to find a $R$ that the verifying algorithm will use to go through the attacked leaf.

In [ ]:
m_fake = b'The earth is flat!'

# Here for this hypertree and this fake message, we know that if R = b'a\x06', we will go through the same leaf on d-1 tree
r = b'a\x06'


This value has been easily found because we can still sign our own message with the secret key (we could then obtain the message signature: b'a\x06\xe1\xe6\xea\xfb\xbd)\xd0+\x9b\xd7+\x94\xd8Vk\xcct\x9fzl\xf1\xe4\xb1%\xcfD\xf4\xa5\xa8\x1e\x14\xa7#\x1a\xe2\xef\xd9\x08\x1f\x8d\xc3?\xee\xfe\xb0\xa0puV\xb6OEH;\xfe\xe1'). But knowing which path is used to verify the signature is publicly known: it depends from $pk$, $m$ and $R$ which are publicly known when we digest the message in the first verification step.

From these informations, we need to compute the $m_{fake}$ signature with different sub-hypertrees (size $h - h'$ with $d-1$ layers) until we can found a tree for which his computed root can be signed with the known $WOTS^+$ leaf. The more faulted signature we have, the more possible roots can be forged. 

From this part, we have to brute force sub-hypertrees possibilities until we found one that can be used with this $WOTS^+$ key. We can then sign our message with our $R$, our tree and we then add the final official authentification path to the signature to make it computable by others.